# Zenodo API upload test

See https://developers.zenodo.org/#quickstart-upload for details

This section walks through all of the steps for upload.


In [71]:
import requests
from pathlib import Path
import json

subdomain = 'https://sandbox.zenodo.org/' # use for testing
#subdomain = 'https://zenodo.org/' # use for production

def load_credential(filename, directory):
    cred = ''
    # to change the script to look for the credential in the working directory, change the value of home to empty string
    if directory == 'home':
        home = str(Path.home()) #gets path to home directory; works for both Win and Mac
        credential_path = home + '/' + filename
    else:
        directory = 'working'
        credential_path = filename
    try:
        with open(credential_path, 'rt', encoding='utf-8') as file_object:
            cred = file_object.read()
    except:
        print(filename + ' file not found - is it in your ' + directory + ' directory?')
        exit()
    return(cred)


In [72]:
if subdomain == 'https://sandbox.zenodo.org/':
    filename = 'zenodo_sandbox_credentials.txt'
elif subdomain == 'https://zenodo.org/':
    filename = 'zenodo_api_credentials.txt'
else:
    print('not a valid subdomain')

access_token = load_credential(filename, 'home')


## Test access by looking at existing depositions

In [75]:
r = requests.get(subdomain + 'api/deposit/depositions',
                 params={'access_token': access_token})
print(r.status_code)


200


In [76]:
print(json.dumps(r.json(), indent = 2))

[
  {
    "conceptdoi": "10.5072/zenodo.750139",
    "conceptrecid": "750139",
    "created": "2021-03-22T01:12:20.480541",
    "doi": "10.5072/zenodo.750140",
    "doi_url": "https://doi.org/10.5072/zenodo.750140",
    "files": [
      {
        "checksum": "c7c6f0ca24f2eff44b278063e7eff034",
        "filename": "gpivi2-co2yr-fem-unopened13499.jpg",
        "filesize": 234334,
        "id": "02868314-bb2a-4c9e-9659-c5de5360e2f5",
        "links": {
          "download": "https://sandbox.zenodo.org/api/files/b95bb29b-4d60-42e1-865d-468c81ba1fcb/gpivi2-co2yr-fem-unopened13499.jpg",
          "self": "https://sandbox.zenodo.org/api/deposit/depositions/750140/files/02868314-bb2a-4c9e-9659-c5de5360e2f5"
        }
      }
    ],
    "id": 750140,
    "links": {
      "badge": "https://sandbox.zenodo.org/badge/doi/10.5072/zenodo.750140.svg",
      "bucket": "https://sandbox.zenodo.org/api/files/4ed1aa5b-e05a-40c3-b335-5fbe24ea289c",
      "conceptbadge": "https://sandbox.zenodo.org/badge/doi

## Create empty upload


In [77]:
headers = {"Content-Type": "application/json"}
params={'access_token': access_token}
r = requests.post(subdomain + 'api/deposit/depositions',
                   params=params,
                   json={},
                   # Headers are not necessary here since "requests" automatically
                   # adds "Content-Type: application/json", because we're using
                   # the "json=" keyword argument
                   # headers=headers,
                   headers=headers)
r.status_code

201

In [78]:
print(json.dumps(r.json(), indent = 2))

{
  "conceptrecid": "750175",
  "created": "2021-03-22T02:30:37.379450+00:00",
  "files": [],
  "id": 750176,
  "links": {
    "bucket": "https://sandbox.zenodo.org/api/files/5b95c598-94f9-44d0-b689-297d1c28294b",
    "discard": "https://sandbox.zenodo.org/api/deposit/depositions/750176/actions/discard",
    "edit": "https://sandbox.zenodo.org/api/deposit/depositions/750176/actions/edit",
    "files": "https://sandbox.zenodo.org/api/deposit/depositions/750176/files",
    "html": "https://sandbox.zenodo.org/deposit/750176",
    "latest_draft": "https://sandbox.zenodo.org/api/deposit/depositions/750176",
    "latest_draft_html": "https://sandbox.zenodo.org/deposit/750176",
    "publish": "https://sandbox.zenodo.org/api/deposit/depositions/750176/actions/publish",
    "self": "https://sandbox.zenodo.org/api/deposit/depositions/750176"
  },
  "metadata": {
    "prereserve_doi": {
      "doi": "10.5072/zenodo.750176",
      "recid": 750176
    }
  },
  "modified": "2021-03-22T02:30:37.37945

## upload file

Upload file using "new" API which supports file sizes with no limit. Upload using HTTP PUT request.  The bucket is a folder-like object storing the files of the record.

In [79]:
bucket_url = r.json()["links"]["bucket"]
deposition_id = r.json()["id"]
filename = "gpivi2-codevel-female13497.jpg"
path = filename

# The target URL is a combination of the bucket link with the desired filename
# seperated by a slash.
with open(path, "rb") as fp:
    r = requests.put(
        "%s/%s" % (bucket_url, filename),
        data=fp,
        params=params,
    )
print(r.status_code)


200


In [80]:
print(json.dumps(r.json(), indent = 2))

{
  "mimetype": "image/jpeg",
  "updated": "2021-03-22T02:31:45.478228+00:00",
  "links": {
    "self": "https://sandbox.zenodo.org/api/files/5b95c598-94f9-44d0-b689-297d1c28294b/gpivi2-codevel-female13497.jpg",
    "version": "https://sandbox.zenodo.org/api/files/5b95c598-94f9-44d0-b689-297d1c28294b/gpivi2-codevel-female13497.jpg?versionId=cce94505-3d5b-4e38-afc6-ff06198ccab5",
    "uploads": "https://sandbox.zenodo.org/api/files/5b95c598-94f9-44d0-b689-297d1c28294b/gpivi2-codevel-female13497.jpg?uploads"
  },
  "is_head": true,
  "created": "2021-03-22T02:31:45.466164+00:00",
  "checksum": "md5:840077d4738c67626949bcea008b0b5e",
  "version_id": "cce94505-3d5b-4e38-afc6-ff06198ccab5",
  "delete_marker": false,
  "key": "gpivi2-codevel-female13497.jpg",
  "size": 157075
}


## Add metadata

See https://developers.zenodo.org/#representation for deposition metadata fields


In [82]:
data = {
     'metadata': {
         'title': 'Pinus virginiana unopened second year female cone',
         'upload_type': 'image',
         'image_type': 'photo',
         'description': 'Test image of an unopened mature cone from a Virginia pine tree.',
         'creators': [{'name': 'Baskauf, Steven',
                       'affiliation': 'Vanderbilt Libraries',
                       'orcid': '0000-0003-4365-3135'}],
         'custom': {
             "dwc:genus": [
              "Pinus"
            ],
            "dwc:specificEpithet": [
              "virginiana"
            ],
            "dwc:taxonRank": [
              "species"
            ]             
         }
     }
 }
r = requests.put(subdomain + 'api/deposit/depositions/%s' % deposition_id,
                  params={'access_token': access_token}, data=json.dumps(data),
                  headers=headers)
print(r.status_code)


200


In [83]:
print(json.dumps(r.json(), indent = 2))

{
  "conceptrecid": "750175",
  "created": "2021-03-22T02:30:37.379450+00:00",
  "doi": "",
  "doi_url": "https://doi.org/",
  "files": [
    {
      "checksum": "840077d4738c67626949bcea008b0b5e",
      "filename": "gpivi2-codevel-female13497.jpg",
      "filesize": 157075,
      "id": "ea7a492a-fc24-4780-90c0-f3a9763af0e1",
      "links": {
        "download": "https://sandbox.zenodo.org/api/files/5b95c598-94f9-44d0-b689-297d1c28294b/gpivi2-codevel-female13497.jpg",
        "self": "https://sandbox.zenodo.org/api/deposit/depositions/750176/files/ea7a492a-fc24-4780-90c0-f3a9763af0e1"
      }
    }
  ],
  "id": 750176,
  "links": {
    "bucket": "https://sandbox.zenodo.org/api/files/5b95c598-94f9-44d0-b689-297d1c28294b",
    "discard": "https://sandbox.zenodo.org/api/deposit/depositions/750176/actions/discard",
    "edit": "https://sandbox.zenodo.org/api/deposit/depositions/750176/actions/edit",
    "files": "https://sandbox.zenodo.org/api/deposit/depositions/750176/files",
    "html":

## Publish

Last step is the publish step

I don't think you can change the metadata after this unless you create a new version.

In [84]:
r = requests.post(subdomain + 'api/deposit/depositions/%s/actions/publish' % deposition_id,
                      params={'access_token': access_token} )
print(r.status_code)

202


In [85]:
print(json.dumps(r.json(), indent = 2))

{
  "conceptdoi": "10.5072/zenodo.750175",
  "conceptrecid": "750175",
  "created": "2021-03-22T02:30:37.379450+00:00",
  "doi": "10.5072/zenodo.750176",
  "doi_url": "https://doi.org/10.5072/zenodo.750176",
  "files": [
    {
      "checksum": "840077d4738c67626949bcea008b0b5e",
      "filename": "gpivi2-codevel-female13497.jpg",
      "filesize": 157075,
      "id": "ea7a492a-fc24-4780-90c0-f3a9763af0e1",
      "links": {
        "download": "https://sandbox.zenodo.org/api/files/5b95c598-94f9-44d0-b689-297d1c28294b/gpivi2-codevel-female13497.jpg",
        "self": "https://sandbox.zenodo.org/api/deposit/depositions/750176/files/ea7a492a-fc24-4780-90c0-f3a9763af0e1"
      }
    }
  ],
  "id": 750176,
  "links": {
    "badge": "https://sandbox.zenodo.org/badge/doi/10.5072/zenodo.750176.svg",
    "bucket": "https://sandbox.zenodo.org/api/files/5b95c598-94f9-44d0-b689-297d1c28294b",
    "conceptbadge": "https://sandbox.zenodo.org/badge/doi/10.5072/zenodo.750175.svg",
    "conceptdoi": "ht

# Download test

Use the List method to get open access records


In [69]:
response = requests.get(subdomain + 'api/records',
                        params={'q': 'Pinus virginiana',
                                'access_token': access_token})


{'aggregations': {'access_right': {'buckets': [{'doc_count': 144, 'key': 'open'}, {'doc_count': 1, 'key': 'closed'}], 'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0}, 'file_type': {'buckets': [{'doc_count': 98, 'key': 'xml'}, {'doc_count': 94, 'key': 'html'}, {'doc_count': 36, 'key': 'pdf'}, {'doc_count': 10, 'key': 'jpg'}, {'doc_count': 1, 'key': 'csv'}, {'doc_count': 1, 'key': 'png'}, {'doc_count': 1, 'key': 'xls'}], 'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0}, 'keywords': {'buckets': [{'doc_count': 96, 'key': 'Animalia'}, {'doc_count': 96, 'key': 'Arthropoda'}, {'doc_count': 94, 'key': 'Arachnida'}, {'doc_count': 94, 'key': 'Araneae'}, {'doc_count': 24, 'key': 'Linyphiidae'}, {'doc_count': 12, 'key': 'Gnaphosidae'}, {'doc_count': 11, 'key': 'Haplodrassus'}, {'doc_count': 8, 'key': 'Thomisidae'}, {'doc_count': 7, 'key': 'Araneidae'}, {'doc_count': 7, 'key': 'Pinus'}], 'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 598}, 'type': {'buckets': [{'doc

In [70]:
print(json.dumps(response.json(), indent=2))

{
  "aggregations": {
    "access_right": {
      "buckets": [
        {
          "doc_count": 144,
          "key": "open"
        },
        {
          "doc_count": 1,
          "key": "closed"
        }
      ],
      "doc_count_error_upper_bound": 0,
      "sum_other_doc_count": 0
    },
    "file_type": {
      "buckets": [
        {
          "doc_count": 98,
          "key": "xml"
        },
        {
          "doc_count": 94,
          "key": "html"
        },
        {
          "doc_count": 36,
          "key": "pdf"
        },
        {
          "doc_count": 10,
          "key": "jpg"
        },
        {
          "doc_count": 1,
          "key": "csv"
        },
        {
          "doc_count": 1,
          "key": "png"
        },
        {
          "doc_count": 1,
          "key": "xls"
        }
      ],
      "doc_count_error_upper_bound": 0,
      "sum_other_doc_count": 0
    },
    "keywords": {
      "buckets": [
        {
          "doc_count": 96,
          "k